In [73]:
import pandas as pd 
import numpy as np
import re
import os
import argparse
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import SimpleRNN , Dropout ,LSTM,Conv1D, GlobalMaxPooling1D, MaxPooling1D , SpatialDropout1D,Activation,Dense, Dropout, Activation, Flatten, Input, concatenate , GRU
from keras.layers.embeddings import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [74]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('arabic')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahmoud\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahmoud\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [108]:
dataset1 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\dataarabTOx.csv')
dataset6 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\Book1.csv')
new = dataset6.dropna()
# print(new.iloc[10])
# print(new.iloc[9])
dataset7 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\Book2.csv')
dataset = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\toxic arabic tweets classification - Copy.tsv' , delimiter = '\t' , quoting = 3)
dataset2 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\OSACT2020-sharedTask-dev.tsv' , delimiter = '\t' , quoting = 3)
dataset3 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\OSACT2020-sharedTask-train.tsv' , delimiter = '\t' , quoting = 3)
dataset4 =dataset2.iloc[:,0:2]
dataset5 =dataset3.iloc[:,0:2]
df_concat = pd.concat([dataset,dataset5,dataset4])
for i in range(0,len(df_concat)):
    if df_concat['Class'].iloc[i] == 'normal' or df_concat['Class'].iloc[i] == 'NOT_OFF' or df_concat['Class'].iloc[i] == 'Non-Offensive' or df_concat['Class'].iloc[i] == 0:
        df_concat['Class'].iloc[i] = 'N'
    elif df_concat['Class'].iloc[i] == 'abusive' or df_concat['Class'].iloc[i] == 'hate' or df_concat['Class'].iloc[i] == 'OFF' or df_concat['Class'].iloc[i] == 'Offensive' or df_concat['Class'].iloc[i] == -1 or df_concat['Class'].iloc[i] == -2:
        df_concat['Class'].iloc[i] = 'P'
print(df_concat.shape)

(15998, 2)


In [122]:
dataset = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\toxic arabic tweets classification.tsv' , delimiter = '\t' , quoting = 3)
df = pd.concat([dataset])
print(df.shape)

(7364, 2)


In [123]:
df.head(10)

,Tweet,Class
0,الوزير جبران باسيل تاج راسك يا جربان ممنوع بعد...,abusive
1,صديقي انت ابن جامعه اللعبه اكبر من داعش اللعبه...,normal
2,و مصلحة لبنان تبدأ باستخراج النفط و الغاز لوقف...,normal
3,كسمك يبن الشرموطه انت و الشرموطة امك يبن الاحبة,abusive
4,كسمين امك يبن كوم الزواني انت و الى جابك,abusive
5,يا جبران باسيل يا معلم يا ريس يا استاذ بدك حضن...,normal
6,ما سمعنا جعاركم مبارح على القصف الإسرائيلي يلي...,hate
7,نصيحة احكي مع الرئيس ميشال عون او هاجم جبران ب...,normal
8,يغبي يبن الغبى يحرق كسمين امك,abusive
9,يبن الشرموطة يبن الشرموطه يحرق كسمينك,abusive


In [109]:
X = df_concat.iloc[: , 0].values
y = df_concat.iloc[: , -1].values

In [110]:
labelencoder_y=LabelEncoder()
y = labelencoder_y.fit_transform(y)
dataset["label"] = labelencoder_y.fit_transform(dataset["Class"])
for i in range(0,len(y)):
    if y[i]==0 or y[i]==1:
        continue
    else:
        y[i]=0
print(max(y))
z = 0
p =0
for i in range(0,len(y)):
    if y[i]==0:
        z=z+1
    elif y[i]==1:
        p=p+1
print(z,' ',p)

1
10389   5609


In [111]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42,shuffle = True)

In [112]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\u2066"
        u"\u2069"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', str(data))

def to_lowercase(text):
    return text.lower()

def remove_eng(text):
    return re.sub(r'[A-Za-z]','',str(text))
    
import string
def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):

    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_emojis(text)
    text = remove_eng(text)
    text = remove_punctuation(text)
    text = remove_punctuations(text)
    text = normalize_arabic(text)
    text = remove_diacritics(text)
    text = remove_repeating_char(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

def normalize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahmoud\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [113]:
X_train = normalize_corpus(X_train)
X_test = normalize_corpus(X_test)

In [114]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,3), tokenizer=text2words,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [115]:
X_train = vec.fit_transform(X_train)
X_test = vec.transform(X_test)

In [116]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
classifier1 = SVC(kernel = 'rbf',C = 10000,probability=False)
classifier1.fit(X_train , y_train)

y_pred1 = classifier1.predict(X_test)
score = accuracy_score(y_test,y_pred1)
print(score*100)

87.28125


In [117]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test , y_pred1)
cm

array([[2008,   80],
       [ 327,  785]], dtype=int64)

In [118]:
import keras
while True:
        
        inp = input("User: ")
        
        if inp.lower() == "quit":
            break
        else:
            result = classifier1.predict(vec.transform(normalize_corpus([inp])))
            print(result)
            if result == 0:
                print("normal")
            else:
                print("abusive")

User: hggi 
[0]
normal
User: 
[0]
normal
User: الله اكبر
[0]
normal
User: الله
[0]
normal
User: الحمد لله
[0]
normal
User: يسكسمك
[0]
normal
User: كسمك
[1]
abusive
User: انت بتقول ايه؟؟
[0]
normal
User: بس يلا
[0]
normal
User: يشرموط
[1]
abusive
User: الله يرحمه ويسكنه فسيح جناته
[0]
normal
User: يارب
[0]
normal
User: رب
[0]
normal
User: الحمدلله ع كل شئ
[0]
normal
User: ضعه
[0]
normal
User: quit


In [119]:
import pickle

# to save the fitted model
with open('SVM_model0.pickle', 'wb') as model:
    pickle.dump(classifier1, model, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted tokenizer
with open('vec0.pickle', 'wb') as ve:
    pickle.dump(vec, ve, protocol=pickle.HIGHEST_PROTOCOL)

In [88]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer(oov_token='UNK')
tok.fit_on_texts(X_train + X_test)
X_train = tok.texts_to_matrix(X_train , mode='tfidf')
X_test = tok.texts_to_matrix(X_test, mode='tfidf')
from sklearn.preprocessing import StandardScaler
sm = StandardScaler()
X_train = sm.fit_transform(X_train)
X_test = sm.transform(X_test)

In [89]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
classifier1 = SVC(kernel = 'rbf',C = 10000)
classifier1.fit(X_train , y_train)

y_pred1 = classifier1.predict(X_test)
score = accuracy_score(y_test,y_pred1)
print(score*100)

79.1875
